In [2]:
using Plots, Gurobi, CSV, DataFrames

In [7]:
"""
    load_repo_data(repo_path::String)

Recursively walks through the specified repository path, identifies all `.csv` and `.txt` files
(even in subfolders), and parses them into a Dictionary of DataFrames.

# Arguments
- `repo_path::String`: The local path to the cloned repository.

# Returns
- `Dict{String, DataFrame}`: A dictionary where keys are unique filenames and values are DataFrames.
"""
function load_repo_data(repo_path::String)
    # Dictionary to store the parsed data
    data_store = Dict{String, DataFrame}()
    
    # CSV options (assume headers exist)
    csv_options = (header=true, stringtype=String)

    if !isdir(repo_path)
        @error "Directory not found: $repo_path"
        return data_store
    end

    @info "Recursively scanning directory: $repo_path"
    
    files_found = 0
    
    # walkdir allows us to search subdirectories (e.g., /data, /src)
    for (root, dirs, files) in walkdir(repo_path)
        for file in files
            # Check for valid extensions
            if endswith(lowercase(file), ".csv") || endswith(lowercase(file), ".txt")
                
                files_found += 1
                full_path = joinpath(root, file)
                dataset_name = splitext(file)[1]
                
                # Handle duplicate filenames in different folders by appending parent folder name
                if haskey(data_store, dataset_name)
                    parent_folder = basename(root)
                    dataset_name = "$(parent_folder)_$(dataset_name)"
                end

                try
                    @info "Parsing: $full_path"
                    df = CSV.read(full_path, DataFrame; csv_options...)
                    data_store[dataset_name] = df
                catch e
                    # Only warn, don't crash, if a file is malformed
                    @warn "Skipping $file: Unable to parse as CSV table."
                end
            end
        end
    end

    if files_found == 0
        @warn "No CSV or TXT files were found in $repo_path or its subdirectories."
        @info "Current working directory contains: $(readdir(repo_path))"
    else
        @info "Successfully loaded $(length(data_store)) datasets."
    end

    return data_store
end
"""
    summarize_data(data::Dict{String, DataFrame})

Prints a brief summary of the loaded datasets.
"""
function summarize_data(data::Dict{String, DataFrame})
    println("\n--- Data Summary ---")
    for (name, df) in data
        println("Dataset: '$name'")
        println("  Shape: $(nrow(df)) rows × $(ncol(df)) columns")
        println("  Cols:  $(join(names(df), ", "))")
        println("--------------------")
    end
end

repo_path = "." 

# 2. Load the data

subway_data = load_repo_data(repo_path)

# 3. Print summary
summarize_data(subway_data)



[ Info: Recursively scanning directory: .
[ Info: Parsing: .\datasets\MTA_Subway_Hourly_Ridership__Oct_21_2024_Evening.csv
[ Info: Parsing: .\datasets\MTA_Subway_Hourly_Ridership__Oct_21_2024_Morning.csv
[ Info: Parsing: .\datasets\MTA_Subway_Stations_20251204.csv
[ Info: Parsing: .\datasets\agency.txt
[ Info: Parsing: .\datasets\calendar.txt
[ Info: Parsing: .\datasets\calendar_dates.txt
[ Info: Parsing: .\datasets\linecapacity.csv
[ Info: Parsing: .\datasets\linelength.csv
[ Info: Parsing: .\datasets\routes.txt
[ Info: Parsing: .\datasets\shapes.txt
[ Info: Parsing: .\datasets\stop_times.txt



--- Data Summary ---
Dataset: 'routes'
  Shape: 29 rows × 10 columns
  Cols:  route_id, agency_id, route_short_name, route_long_name, route_desc, route_type, route_url, route_color, route_text_color, route_sort_order
--------------------
Dataset: 'nodes_with_ridership'
  Shape: 475 rows × 9 columns
  Cols:  node_idx, stop_id, stop_name, stop_lon, stop_lat, station_complex_id, ridership_morning, ridership_evening, net_ridership
--------------------
Dataset: 'stops'
  Shape: 1488 rows × 6 columns
  Cols:  stop_id, stop_name, stop_lat, stop_lon, location_type, parent_station
--------------------
Dataset: 'trips'
  Shape: 20304 rows × 6 columns
  Cols:  route_id, trip_id, service_id, trip_headsign, direction_id, shape_id
--------------------
Dataset: 'MTA_Subway_Aggregated_Ridership_Oct_21_2024_Evening'
  Shape: 424 rows × 2 columns
  Cols:  station_complex_id, ridership
--------------------
Dataset: 'agency'
  Shape: 1 rows × 6 columns
  Cols:  agency_id, agency_name, agency_url, agency_

[ Info: Parsing: .\datasets\stops.txt
[ Info: Parsing: .\datasets\transfers.txt
[ Info: Parsing: .\datasets\trips.txt
[ Info: Parsing: .\generated_graphs\edges_by_route.csv
[ Info: Parsing: .\generated_graphs\nodes.csv
[ Info: Parsing: .\generated_graphs\nodes_with_ridership.csv
[ Info: Parsing: .\generated_graphs\routes.csv
[ Info: Parsing: .\generated_graphs\stop_routes.csv
[ Info: Parsing: .\generated_turnstile_data\MTA_Subway_Aggregated_Ridership_Oct_21_2024_Evening.csv
[ Info: Parsing: .\generated_turnstile_data\MTA_Subway_Aggregated_Ridership_Oct_21_2024_Morning.csv
[ Info: Parsing: .\generated_turnstile_data\evening_4to8_with_gtfs.csv
[ Info: Parsing: .\generated_turnstile_data\morning_6to10_with_gtfs.csv
[ Info: Successfully loaded 23 datasets.
